In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
import math
import os
import json
import time
import seaborn as sns
import datetime
from core.data_processor import DataLoader
from core.model import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
'''
till now judgement
date_time: convert to suitable format, doesnt go for prediction 
dayofweek: add new column signifying day(Monday=0, Sunday=6)
peakhours: 
is_holiday : change all non null to 1
wind_speed : to be standardized variable
wind_direction: categorical variable, better to put in 60 deg buckets 
visibility_in_miles: categorical
dew_point: categorical
temperature: replace 0 with mean value and standardize it
air_pollution_index : No change, just standardize it
humidity: standardize it
rain_p_h: have to think , for now just use as it is
snow_ph: 0 and 1
weather_type: label encode
weather_description: for now label encode
clouds_all : 
traffic_volume: to predict
'''


'\ntill now judgement\ndate_time: convert to suitable format, doesnt go for prediction \ndayofweek: add new column signifying day(Monday=0, Sunday=6)\npeakhours: \nis_holiday : change all non null to 1\nwind_speed : to be standardized variable\nwind_direction: categorical variable, better to put in 60 deg buckets \nvisibility_in_miles: categorical\ndew_point: categorical\ntemperature: replace 0 with mean value and standardize it\nair_pollution_index : No change, just standardize it\nhumidity: standardize it\nrain_p_h: have to think , for now just use as it is\nsnow_ph: 0 and 1\nweather_type: label encode\nweather_description: for now label encode\nclouds_all : \ntraffic_volume: to predict\n'

In [3]:
rawdata = pd.read_csv("./DataSets/Train.csv")
df_test = pd.read_csv("./DataSets/Test.csv")

In [4]:
rawdata.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [5]:
rawdata.date_time = pd.to_datetime(rawdata['date_time'])

In [6]:
# df['CumulativeRain'] = rawdata['rain_p_h'].rolling(3).sum()
# df['CumulativeSnow'] = rawdata['snow_p_h'].rolling(12).sum()
# df.CumulativeRain.fillna(0)
# df.CumulativeSnow.fillna(0)
# df['isRain'] = (rawdata['rain_p_h']>0)*1
# df['isSnow'] = (rawdata['snow_p_h']>0)*1
# df['isWorkingHour'] = (rawdata['datetime'].dt.time>datetime.time(6,0,0)) & (rawdata['datatime'].dt.time<datetime.time(19,0,0))*1
# df.drop(columns=['dew_point'],inplace=True)
# # Data.head()

# HolidayDates = rawdata.loc[rawdata['is_holiday']!='None']['date']
# df.loc[rawdata.loc[rawdata['datetime'].isin(HolidayDates)]['is_holiday'].index,'is_holiday'] = Data.loc[Data.loc[Data['date'].isin(HolidayDates)]['is_holiday'].index,'is_holiday'].replace('None',float('NaN')).fillna(method='ffill').values
# (((HolidayDates - pd.DateOffset(1)).dt.weekday_name == 'Sunday').index - pd.DateOffset(1))
# (((HolidayDates + pd.DateOffset(1)).dt.weekday_name == 'Saturday').index + pd.DateOffset(1))

In [7]:
rawdata.groupby(['weather_type','weather_description']).mean()[['clouds_all','traffic_volume']]

clouds_all  traffic_volume
weather_type weather_description                                            
Clear        Sky is Clear                           0.000000     3417.205968
             sky is clear                           1.823498     3026.460393
Clouds       broken clouds                         71.264575     3481.830719
             few clouds                            19.151148     3569.742347
             overcast clouds                       90.191843     3302.797080
             scattered clouds                      39.897123     3841.502616
Drizzle      drizzle                               87.136957     3041.732609
             heavy intensity drizzle               88.650000     3158.500000
             light intensity drizzle               82.244666     3309.857752
             shower drizzle                        90.000000     2010.000000
Fog          fog                                   54.710758     2740.603175
Haze         haze                                  60.702882     3638.919069
Mist         mist                                  69.286385     2943.952310
Rain         freezing rain                         90.000000     4314.000000
             heavy intensity rain                  79.479100     3069.070740
             light intensity shower rain           90.000000     4667.500000
             light rain                            70.572072     3369.970925
             moderate rain                         80.818100     3113.825339
             proximity shower rain                 73.018519     4491.685185
             very heavy rain                       58.500000     2568.833333
Smoke        smoke                                 53.882353     2936.176471
Snow         heavy snow                            86.010221     3085.862010
             light rain and snow                   83.166667     3961.166667
             light shower snow                     67.750000     4570.750000
             light snow                            78.450000     3031.460938
             shower snow                           90.000000     5664.000000
             sleet                                 90.000000     3882.000000
             snow                                  87.200000     2695.905263
Squall       SQUALLS                               76.000000     2061.750000
Thunderstorm proximity thunderstorm                67.776812     2992.915942
             proximity thunderstorm with drizzle   84.272727     2810.090909
             proximity thunderstorm with rain      82.200000     2076.520000
             thunderstorm                          64.924242     2558.833333
             thunderstorm with drizzle             90.000000     2297.000000
             thunderstorm with heavy rain          74.952381     2544.619048
             thunderstorm with light drizzle       72.000000     2858.750000
             thunderstorm with light rain          78.571429     2501.857143
             thunderstorm with rain                84.739130     3164.695652

In [14]:
def preprocessing(rawdata):
    df = rawdata.copy()
    #seasons
    df['month'] = rawdata.date_time.dt.month
    df['season'] = 0
    df.loc[(df.month==11) |(df.month==12)|(df.month==1) |(df.month==2),'season'] = 1  
    #working hours
    df['isWorkingHour'] = ((rawdata['date_time'].dt.time>datetime.time(6,0,0)) & (rawdata['date_time'].dt.time<datetime.time(19,0,0)))*1
    #is_holiday
    df['is_holiday'] = (rawdata.is_holiday!='None')*1
    A = df.loc[df.is_holiday==1,'date_time']
    for items in A:
        df.loc[(df.date_time.dt.year==items.year) & (df.date_time.dt.month==items.month) & (df.date_time.dt.day==items.day),'is_holiday'] = 1
    df.loc[(rawdata.date_time.dt.weekday_name=='Saturday') | (rawdata.date_time.dt.weekday_name=='Sunday'),'is_holiday'] = 1
    #wind_direction
    bins = [-0.1, 60, 120, 180, 240, 300, 360]
    labels = [0,1,2,3,4,5]
    df['wind_direction']= pd.cut(rawdata['wind_direction'], bins=bins, labels=labels)
    #temperature
    df.loc[rawdata.temperature==0,'temperature'] = np.mean(rawdata.loc[rawdata.temperature!=0,'temperature'])
    #Cumulative variables
    df['CumulativeRain'] = rawdata['rain_p_h'].rolling(3).sum()
    df['CumulativeSnow'] = rawdata['snow_p_h'].rolling(12).sum()
    df.CumulativeRain.fillna(0,inplace=True)
    df.CumulativeSnow.fillna(0,inplace=True)
    #Rain snow indicator
    df['isRain'] = (rawdata['rain_p_h']>0)*1
    df['isSnow'] = (rawdata['snow_p_h']>0)*1
    #weather label encode
    df.loc[rawdata.weather_type=='Clouds','weather_type'] = 'Clear'
    df.loc[rawdata.weather_type=='Rain','weather_type'] = rawdata.loc[rawdata.weather_type=='Rain','weather_description']
    df.loc[rawdata.weather_type=='Thunderstorm','weather_type'] = rawdata.loc[rawdata.weather_type=='Thunderstorm','weather_description']
    #Encoding features
    lb_weather = LabelEncoder()
    df['weather_type'] = lb_weather.fit_transform(rawdata['weather_type'])
    #standardizing
    sc = MinMaxScaler()
    std_cols = ['temperature','air_pollution_index','humidity','wind_speed','clouds_all']
    sc_traffic = MinMaxScaler()
    df["traffic_volume"] = sc_traffic.fit_transform(np.array(rawdata["traffic_volume"]).reshape(-1,1))
    df[std_cols] = sc.fit_transform(rawdata[std_cols])
    #dropping columns
    df.drop(columns=['dew_point','month','rain_p_h','snow_p_h','weather_description'],inplace=True)
    return df,lb_weather,(std_cols,sc,sc_traffic)

In [15]:
df,encoders,stats = preprocessing(rawdata)

In [16]:
df.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,temperature,clouds_all,weather_type,traffic_volume,season,isWorkingHour,CumulativeRain,CumulativeSnow,isRain,isSnow
0,2012-10-02 09:00:00,0,0.384083,0.873563,0.1250,5,1,0.935245,0.40,1,0.761676,0,1,0.0,0.0,0,0
1,2012-10-02 10:00:00,0,0.581315,0.620690,0.1875,5,1,0.938749,0.75,1,0.620330,0,1,0.0,0.0,0,0
2,2012-10-02 11:00:00,0,0.356401,0.609195,0.1875,5,2,0.939463,0.90,1,0.654808,0,1,0.0,0.0,0,0
3,2012-10-02 12:00:00,0,0.034602,0.609195,0.1875,5,5,0.941247,0.90,1,0.690385,0,1,0.0,0.0,0,0
4,2012-10-02 13:00:00,0,0.937716,0.597701,0.1875,5,7,0.944524,0.75,1,0.675549,0,1,0.0,0.0,0,0


In [18]:
len(df.columns)

17

In [10]:
#saving file for easy training purpose
print('Saving as CSV...')
df.to_csv('clean_train.csv',index=False)
df.info()

Saving as CSV...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 18 columns):
date_time              33750 non-null datetime64[ns]
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null float64
humidity               33750 non-null float64
wind_speed             33750 non-null float64
wind_direction         33750 non-null category
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null int64
clouds_all             33750 non-null float64
weather_type           33750 non-null int64
weather_description    33750 non-null int64
traffic_volume         33750 non-null float64
dayofweek              33750 non-null int64
month                  33750 non-null int64
hour                   33750 non-null category
dtypes: category(2), datetime64[ns](1), float64(7), i

In [11]:
#running model
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

data = DataLoader(
    configs['data']['filename'],
    configs['data']['train_test_split'],
    configs['data']['columns']
)

model = Model()
model.build_model(configs)
# model.load_model('./saved_models/28072019-004141-e50.h5')

# out-of memory generative training
steps_per_epoch = math.ceil((data.len_train - configs['data']['sequence_length']) / configs['training']['batch_size'])
model.train_generator(
    data_gen=data.generate_train_batch(
        seq_len=configs['data']['sequence_length'],
        batch_size=configs['training']['batch_size'],
        normalise=configs['data']['normalise']
    ),
    epochs=configs['training']['epochs'],
    batch_size=configs['training']['batch_size'],
    steps_per_epoch=steps_per_epoch,
    save_dir=configs['model']['save_dir']
)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 30)            5760      
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 40)            11360     
_________________________________________________________________
lstm_3 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 41        
Total params: 30,121
Trainable params: 30,121
Non-trainable params: 0
_________________________________________________________________
[Model

StopIteration: 

In [61]:
#Loading model if you didnt train
#running model
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

data = DataLoader(
    configs['data']['filename'],
    configs['data']['train_test_split'],
    configs['data']['columns']
)

model = Model()
model.build_model(configs)
model.load_model('./saved_models/28072019-225156-e80.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 24, 30)            5760      
_________________________________________________________________
dropout_5 (Dropout)          (None, 24, 30)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 24, 40)            11360     
_________________________________________________________________
lstm_9 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_6 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 41        
Total params: 30,121
Trainable params: 30,121
Non-trainable params: 0
_________________________________________________________________
[Model

In [62]:
#Test data preperation
df_test.date_time = pd.to_datetime(df_test['date_time'])

In [63]:
def prepping_test(rawdata,stats,encoder):
    df = rawdata.copy()
    #day of week, we have less traffic during saturdays or sundays
    df['dayofweek'] = rawdata.date_time.dt.dayofweek
    df['month'] = rawdata.date_time.dt.month
    #hours
    bins = [-0.1,6,12,18,24]
    labels = [0,1,2,3]
    df['hour']= pd.cut(rawdata['date_time'].dt.hour, bins=bins, labels=labels)
    #is_holiday
    df['is_holiday'] = (rawdata.is_holiday!='None')*1
    A = df.loc[df.is_holiday==1,'date_time']
    for items in A:
        df.loc[(df.date_time.dt.year==items.year) & (df.date_time.dt.month==items.month) & (df.date_time.dt.day==items.day),'is_holiday'] = 1
    #wind_direction
    bins = [-0.1, 60, 120, 180, 240, 300, 360]
    labels = [0,1,2,3,4,5]
    df['wind_direction']= pd.cut(rawdata['wind_direction'], bins=bins, labels=labels)
    #temperature
    df.loc[rawdata.temperature==0,'temperature'] = np.mean(rawdata.loc[rawdata.temperature!=0,'temperature'])
    #snow_p_h
    df['snow_p_h'] = (rawdata.snow_p_h!=0)*1
    #weather label encode
    #Encoding features
    df['weather_type'] = encoder[0].transform(rawdata['weather_type'])
    df['weather_description'] = encoder[1].transform(rawdata['weather_description'])
    #normalizing, not including traffic volume
    std_cols = stats[0]
#     mean = stats[1][:-1]
#     std = stats[2][:-1]
    df[std_cols] = stats[1].transform(rawdata[std_cols])
    return df

In [64]:
df_test_prepped = prepping_test(df_test,stats,encoders)
df_test_prepped["traffic_volume"] = np.NaN
data_to_attach=configs["data"]["sequence_length"]-1
df_test_prepped=pd.concat([df.iloc[-data_to_attach:],df_test_prepped],axis=0)
df_test_prepped.reset_index(drop=True,inplace=True)
df_test_prepped.to_csv("./clean_test.csv",index=False)

In [65]:
def next_window(data_train,i, seq_len):
    '''Generates the next data window from the given index location i'''
    window = data_train[i:i+seq_len]
    x = window[:-1]
    y = window[-1, [0]]
    return x, y

In [66]:
#I generate data window, preditict and update and repeat
for i in range(len(df_test_prepped)-configs['data']['sequence_length'] + 1):
    print("Done {}/{}".format(i,len(df_test_prepped)-configs['data']['sequence_length']+1))
    x_test,y = next_window(df_test_prepped.get(configs["data"]["columns"]).values,i,configs['data']['sequence_length'])    
    pred = model.model.predict(x_test[np.newaxis])[0][0]
    df_test_prepped.traffic_volume[configs["data"]["sequence_length"]-1+i] = pred

Done 0/14454
Done 1/14454
Done 2/14454
Done 3/14454
Done 4/14454
Done 5/14454
Done 6/14454
Done 7/14454
Done 8/14454
Done 9/14454
Done 10/14454
Done 11/14454
Done 12/14454
Done 13/14454
Done 14/14454
Done 15/14454
Done 16/14454
Done 17/14454
Done 18/14454
Done 19/14454
Done 20/14454
Done 21/14454
Done 22/14454
Done 23/14454
Done 24/14454
Done 25/14454
Done 26/14454
Done 27/14454
Done 28/14454
Done 29/14454
Done 30/14454
Done 31/14454
Done 32/14454
Done 33/14454
Done 34/14454
Done 35/14454
Done 36/14454
Done 37/14454
Done 38/14454
Done 39/14454
Done 40/14454
Done 41/14454
Done 42/14454
Done 43/14454
Done 44/14454
Done 45/14454
Done 46/14454
Done 47/14454
Done 48/14454
Done 49/14454
Done 50/14454
Done 51/14454
Done 52/14454
Done 53/14454
Done 54/14454
Done 55/14454
Done 56/14454
Done 57/14454
Done 58/14454
Done 59/14454
Done 60/14454
Done 61/14454
Done 62/14454
Done 63/14454
Done 64/14454
Done 65/14454
Done 66/14454
Done 67/14454
Done 68/14454
Done 69/14454
Done 70/14454
Done 71/14454
Do

Done 554/14454
Done 555/14454
Done 556/14454
Done 557/14454
Done 558/14454
Done 559/14454
Done 560/14454
Done 561/14454
Done 562/14454
Done 563/14454
Done 564/14454
Done 565/14454
Done 566/14454
Done 567/14454
Done 568/14454
Done 569/14454
Done 570/14454
Done 571/14454
Done 572/14454
Done 573/14454
Done 574/14454
Done 575/14454
Done 576/14454
Done 577/14454
Done 578/14454
Done 579/14454
Done 580/14454
Done 581/14454
Done 582/14454
Done 583/14454
Done 584/14454
Done 585/14454
Done 586/14454
Done 587/14454
Done 588/14454
Done 589/14454
Done 590/14454
Done 591/14454
Done 592/14454
Done 593/14454
Done 594/14454
Done 595/14454
Done 596/14454
Done 597/14454
Done 598/14454
Done 599/14454
Done 600/14454
Done 601/14454
Done 602/14454
Done 603/14454
Done 604/14454
Done 605/14454
Done 606/14454
Done 607/14454
Done 608/14454
Done 609/14454
Done 610/14454
Done 611/14454
Done 612/14454
Done 613/14454
Done 614/14454
Done 615/14454
Done 616/14454
Done 617/14454
Done 618/14454
Done 619/14454
Done 620/1

Done 1094/14454
Done 1095/14454
Done 1096/14454
Done 1097/14454
Done 1098/14454
Done 1099/14454
Done 1100/14454
Done 1101/14454
Done 1102/14454
Done 1103/14454
Done 1104/14454
Done 1105/14454
Done 1106/14454
Done 1107/14454
Done 1108/14454
Done 1109/14454
Done 1110/14454
Done 1111/14454
Done 1112/14454
Done 1113/14454
Done 1114/14454
Done 1115/14454
Done 1116/14454
Done 1117/14454
Done 1118/14454
Done 1119/14454
Done 1120/14454
Done 1121/14454
Done 1122/14454
Done 1123/14454
Done 1124/14454
Done 1125/14454
Done 1126/14454
Done 1127/14454
Done 1128/14454
Done 1129/14454
Done 1130/14454
Done 1131/14454
Done 1132/14454
Done 1133/14454
Done 1134/14454
Done 1135/14454
Done 1136/14454
Done 1137/14454
Done 1138/14454
Done 1139/14454
Done 1140/14454
Done 1141/14454
Done 1142/14454
Done 1143/14454
Done 1144/14454
Done 1145/14454
Done 1146/14454
Done 1147/14454
Done 1148/14454
Done 1149/14454
Done 1150/14454
Done 1151/14454
Done 1152/14454
Done 1153/14454
Done 1154/14454
Done 1155/14454
Done 115

Done 1607/14454
Done 1608/14454
Done 1609/14454
Done 1610/14454
Done 1611/14454
Done 1612/14454
Done 1613/14454
Done 1614/14454
Done 1615/14454
Done 1616/14454
Done 1617/14454
Done 1618/14454
Done 1619/14454
Done 1620/14454
Done 1621/14454
Done 1622/14454
Done 1623/14454
Done 1624/14454
Done 1625/14454
Done 1626/14454
Done 1627/14454
Done 1628/14454
Done 1629/14454
Done 1630/14454
Done 1631/14454
Done 1632/14454
Done 1633/14454
Done 1634/14454
Done 1635/14454
Done 1636/14454
Done 1637/14454
Done 1638/14454
Done 1639/14454
Done 1640/14454
Done 1641/14454
Done 1642/14454
Done 1643/14454
Done 1644/14454
Done 1645/14454
Done 1646/14454
Done 1647/14454
Done 1648/14454
Done 1649/14454
Done 1650/14454
Done 1651/14454
Done 1652/14454
Done 1653/14454
Done 1654/14454
Done 1655/14454
Done 1656/14454
Done 1657/14454
Done 1658/14454
Done 1659/14454
Done 1660/14454
Done 1661/14454
Done 1662/14454
Done 1663/14454
Done 1664/14454
Done 1665/14454
Done 1666/14454
Done 1667/14454
Done 1668/14454
Done 166

Done 2120/14454
Done 2121/14454
Done 2122/14454
Done 2123/14454
Done 2124/14454
Done 2125/14454
Done 2126/14454
Done 2127/14454
Done 2128/14454
Done 2129/14454
Done 2130/14454
Done 2131/14454
Done 2132/14454
Done 2133/14454
Done 2134/14454
Done 2135/14454
Done 2136/14454
Done 2137/14454
Done 2138/14454
Done 2139/14454
Done 2140/14454
Done 2141/14454
Done 2142/14454
Done 2143/14454
Done 2144/14454
Done 2145/14454
Done 2146/14454
Done 2147/14454
Done 2148/14454
Done 2149/14454
Done 2150/14454
Done 2151/14454
Done 2152/14454
Done 2153/14454
Done 2154/14454
Done 2155/14454
Done 2156/14454
Done 2157/14454
Done 2158/14454
Done 2159/14454
Done 2160/14454
Done 2161/14454
Done 2162/14454
Done 2163/14454
Done 2164/14454
Done 2165/14454
Done 2166/14454
Done 2167/14454
Done 2168/14454
Done 2169/14454
Done 2170/14454
Done 2171/14454
Done 2172/14454
Done 2173/14454
Done 2174/14454
Done 2175/14454
Done 2176/14454
Done 2177/14454
Done 2178/14454
Done 2179/14454
Done 2180/14454
Done 2181/14454
Done 218

...

Done 12236/14454
Done 12237/14454
Done 12238/14454
Done 12239/14454
Done 12240/14454
Done 12241/14454
Done 12242/14454
Done 12243/14454
Done 12244/14454
Done 12245/14454
Done 12246/14454
Done 12247/14454
Done 12248/14454
Done 12249/14454
Done 12250/14454
Done 12251/14454
Done 12252/14454
Done 12253/14454
Done 12254/14454
Done 12255/14454
Done 12256/14454
Done 12257/14454
Done 12258/14454
Done 12259/14454
Done 12260/14454
Done 12261/14454
Done 12262/14454
Done 12263/14454
Done 12264/14454
Done 12265/14454
Done 12266/14454
Done 12267/14454
Done 12268/14454
Done 12269/14454
Done 12270/14454
Done 12271/14454
Done 12272/14454
Done 12273/14454
Done 12274/14454
Done 12275/14454
Done 12276/14454
Done 12277/14454
Done 12278/14454
Done 12279/14454
Done 12280/14454
Done 12281/14454
Done 12282/14454
Done 12283/14454
Done 12284/14454
Done 12285/14454
Done 12286/14454
Done 12287/14454
Done 12288/14454
Done 12289/14454
Done 12290/14454
Done 12291/14454
Done 12292/14454
Done 12293/14454
Done 12294/144

Done 12718/14454
Done 12719/14454
Done 12720/14454
Done 12721/14454
Done 12722/14454
Done 12723/14454
Done 12724/14454
Done 12725/14454
Done 12726/14454
Done 12727/14454
Done 12728/14454
Done 12729/14454
Done 12730/14454
Done 12731/14454
Done 12732/14454
Done 12733/14454
Done 12734/14454
Done 12735/14454
Done 12736/14454
Done 12737/14454
Done 12738/14454
Done 12739/14454
Done 12740/14454
Done 12741/14454
Done 12742/14454
Done 12743/14454
Done 12744/14454
Done 12745/14454
Done 12746/14454
Done 12747/14454
Done 12748/14454
Done 12749/14454
Done 12750/14454
Done 12751/14454
Done 12752/14454
Done 12753/14454
Done 12754/14454
Done 12755/14454
Done 12756/14454
Done 12757/14454
Done 12758/14454
Done 12759/14454
Done 12760/14454
Done 12761/14454
Done 12762/14454
Done 12763/14454
Done 12764/14454
Done 12765/14454
Done 12766/14454
Done 12767/14454
Done 12768/14454
Done 12769/14454
Done 12770/14454
Done 12771/14454
Done 12772/14454
Done 12773/14454
Done 12774/14454
Done 12775/14454
Done 12776/144

Done 13200/14454
Done 13201/14454
Done 13202/14454
Done 13203/14454
Done 13204/14454
Done 13205/14454
Done 13206/14454
Done 13207/14454
Done 13208/14454
Done 13209/14454
Done 13210/14454
Done 13211/14454
Done 13212/14454
Done 13213/14454
Done 13214/14454
Done 13215/14454
Done 13216/14454
Done 13217/14454
Done 13218/14454
Done 13219/14454
Done 13220/14454
Done 13221/14454
Done 13222/14454
Done 13223/14454
Done 13224/14454
Done 13225/14454
Done 13226/14454
Done 13227/14454
Done 13228/14454
Done 13229/14454
Done 13230/14454
Done 13231/14454
Done 13232/14454
Done 13233/14454
Done 13234/14454
Done 13235/14454
Done 13236/14454
Done 13237/14454
Done 13238/14454
Done 13239/14454
Done 13240/14454
Done 13241/14454
Done 13242/14454
Done 13243/14454
Done 13244/14454
Done 13245/14454
Done 13246/14454
Done 13247/14454
Done 13248/14454
Done 13249/14454
Done 13250/14454
Done 13251/14454
Done 13252/14454
Done 13253/14454
Done 13254/14454
Done 13255/14454
Done 13256/14454
Done 13257/14454
Done 13258/144

Done 13682/14454
Done 13683/14454
Done 13684/14454
Done 13685/14454
Done 13686/14454
Done 13687/14454
Done 13688/14454
Done 13689/14454
Done 13690/14454
Done 13691/14454
Done 13692/14454
Done 13693/14454
Done 13694/14454
Done 13695/14454
Done 13696/14454
Done 13697/14454
Done 13698/14454
Done 13699/14454
Done 13700/14454
Done 13701/14454
Done 13702/14454
Done 13703/14454
Done 13704/14454
Done 13705/14454
Done 13706/14454
Done 13707/14454
Done 13708/14454
Done 13709/14454
Done 13710/14454
Done 13711/14454
Done 13712/14454
Done 13713/14454
Done 13714/14454
Done 13715/14454
Done 13716/14454
Done 13717/14454
Done 13718/14454
Done 13719/14454
Done 13720/14454
Done 13721/14454
Done 13722/14454
Done 13723/14454
Done 13724/14454
Done 13725/14454
Done 13726/14454
Done 13727/14454
Done 13728/14454
Done 13729/14454
Done 13730/14454
Done 13731/14454
Done 13732/14454
Done 13733/14454
Done 13734/14454
Done 13735/14454
Done 13736/14454
Done 13737/14454
Done 13738/14454
Done 13739/14454
Done 13740/144

Done 14164/14454
Done 14165/14454
Done 14166/14454
Done 14167/14454
Done 14168/14454
Done 14169/14454
Done 14170/14454
Done 14171/14454
Done 14172/14454
Done 14173/14454
Done 14174/14454
Done 14175/14454
Done 14176/14454
Done 14177/14454
Done 14178/14454
Done 14179/14454
Done 14180/14454
Done 14181/14454
Done 14182/14454
Done 14183/14454
Done 14184/14454
Done 14185/14454
Done 14186/14454
Done 14187/14454
Done 14188/14454
Done 14189/14454
Done 14190/14454
Done 14191/14454
Done 14192/14454
Done 14193/14454
Done 14194/14454
Done 14195/14454
Done 14196/14454
Done 14197/14454
Done 14198/14454
Done 14199/14454
Done 14200/14454
Done 14201/14454
Done 14202/14454
Done 14203/14454
Done 14204/14454
Done 14205/14454
Done 14206/14454
Done 14207/14454
Done 14208/14454
Done 14209/14454
Done 14210/14454
Done 14211/14454
Done 14212/14454
Done 14213/14454
Done 14214/14454
Done 14215/14454
Done 14216/14454
Done 14217/14454
Done 14218/14454
Done 14219/14454
Done 14220/14454
Done 14221/14454
Done 14222/144

In [67]:
backup = df_test_prepped.traffic_volume

In [68]:
np.ravel(stats[2].inverse_transform(np.array(backup).reshape(-1,1)))

array([4789.        , 5185.        , 5590.        , ...,  768.8209343 ,
        594.7388792 ,  454.52767611])

In [69]:
#de normalizing 
df_test_prepped.traffic_volume = np.ravel(stats[2].inverse_transform(np.array(df_test_prepped.traffic_volume).reshape(-1,1)))

In [70]:
#Submission
df_submit = df_test_prepped[["date_time","traffic_volume"]]
df_submit = df_submit[24:]
df_submit.to_csv("Submission_v2.csv",index=False)

In [60]:
df_submit

,date_time,traffic_volume
24,2017-05-18 00:00:00,1149.653065
25,2017-05-18 00:00:00,793.425603
26,2017-05-18 00:00:00,685.976799
27,2017-05-18 01:00:00,509.349358
28,2017-05-18 01:00:00,421.166899
29,2017-05-18 02:00:00,458.873837
30,2017-05-18 02:00:00,568.334086
31,2017-05-18 02:00:00,621.914535
32,2017-05-18 03:00:00,864.981914
33,2017-05-18 03:00:00,1070.383794
